In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.impute import SimpleImputer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, recall_score,roc_auc_score
import optuna
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

In [4]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
comb=pd.concat([train,test])

In [6]:
comb.isna().sum()

PassengerId        0
HomePlanet       288
CryoSleep        310
Cabin            299
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Transported     4277
dtype: int64

In [7]:
Expenses_columns = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
comb.loc[:,Expenses_columns]=comb.apply(lambda x: 0 if x.CryoSleep == True else x,axis =1)

In [8]:
comb["in"] = comb["Spa"] + comb["VRDeck"] + comb["RoomService"]

comb["out"] =  comb['FoodCourt'] + comb['ShoppingMall'] 


In [9]:
comb['Expenses'] = comb.loc[:,Expenses_columns].sum(axis=1)

In [11]:
comb.loc[:,['CryoSleep']]=comb.apply(lambda x: True if x.Expenses == 0 and pd.isna(x.CryoSleep) else x,axis =1)

In [13]:
comb.Name = comb.Name.fillna('x x')

In [14]:
comb.loc[:,['Room']] = comb.PassengerId.apply(lambda x: x[0:4] )
guide_VIP=comb.loc[:,['Room','VIP']].dropna().drop_duplicates('Room')
guide_Cabin=comb.loc[:,['Room','Cabin']].dropna().drop_duplicates('Room')
guide_HomePlanet=comb.loc[:,['Room','HomePlanet']].dropna().drop_duplicates('Room')
guide_Destination=comb.loc[:,['Room','Destination']].dropna().drop_duplicates('Room')
comb=pd.merge(comb,guide_Cabin,how="left",on='Room',suffixes=('','_y'))
comb=pd.merge(comb,guide_VIP,how="left",on='Room',suffixes=('','_y'))
comb=pd.merge(comb,guide_HomePlanet,how="left",on='Room',suffixes=('','_y'))
comb=pd.merge(comb,guide_Destination,how="left",on='Room',suffixes=('','_y'))

In [15]:
comb.loc[:,['VIP']]=comb.apply(lambda x: x.VIP_y if pd.isna(x.VIP) else x,axis=1)
comb.loc[:,['Cabin']]=comb.apply(lambda x:  x.Cabin_y if pd.isna(x.Cabin) else x,axis=1)
comb.loc[:,['HomePlanet']]=comb.apply(lambda x:  x.HomePlanet_y if pd.isna(x.HomePlanet) else x,axis=1)
comb.loc[:,['Destination']]=comb.apply(lambda x:  x.Destination_y if pd.isna(x.Destination) else x,axis=1)

In [16]:
comb.loc[:,['Cabin_1']] = comb.Cabin.str.split("/",expand=True).iloc[:,0]
comb.loc[:,['Cabin_2']] = comb.Cabin.str.split("/",expand=True).iloc[:,1]
comb.loc[:,['Cabin_3']] = comb.Cabin.str.split("/",expand=True).iloc[:,2]

In [18]:
num_cols = ['ShoppingMall','FoodCourt','RoomService','Spa','VRDeck','Expenses','Age',"in","out"]
cat_cols = ['CryoSleep','Cabin_1','Cabin_3','VIP','HomePlanet','Destination']
transported=['Transported']
comb = comb[num_cols+cat_cols+transported].copy()

In [20]:
nums = [col for col in comb.columns if comb[col].dtype not in ["O"]]
nums

['ShoppingMall',
 'FoodCourt',
 'RoomService',
 'Spa',
 'VRDeck',
 'Expenses',
 'Age',
 'in',
 'out']

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
imp_missforest = IterativeImputer(
    estimator=ExtraTreesRegressor(),
    max_iter=30
).set_output(transform="pandas")
imp_missforest.fit(train_test[nums])

In [28]:
comb[nums] = imp_missforest.transform(comb[nums])

In [29]:
cat_imp = SimpleImputer(strategy='most_frequent')
comb[cat_cols] = cat_imp.fit_transform(comb[cat_cols])

In [30]:
comb.Cabin_3=comb.Cabin_3.fillna("P")

In [31]:
comb.isnull().sum()

ShoppingMall       0
FoodCourt          0
RoomService        0
Spa                0
VRDeck             0
Expenses           0
Age                0
inline             0
out                0
CryoSleep          0
Cabin_1            0
Cabin_3            0
VIP                0
HomePlanet         0
Destination        0
Transported     4277
dtype: int64

In [32]:
pd.crosstab(comb.HomePlanet,comb.Transported)

Transported,False,True
HomePlanet,,
Earth,2728,2017
Europa,735,1426
Mars,852,935


In [33]:
pd.crosstab(comb.Destination,comb.Transported)

Transported,False,True
Destination,,
55 Cancri e,709,1108
PSO J318.5-22,399,405
TRAPPIST-1e,3207,2865


In [36]:
mask = (comb["HomePlanet"] == "Europa")  & (comb["Destination"]=="55 Cancri e")
mask2 = (comb["HomePlanet"] == "Earth")  & (comb["Destination"]=="TRAPPIST-1e")
mask3 = (comb["HomePlanet"] == "Europa")  & (comb["Destination"]=="55 Cancri e") & (comb["CryoSleep"]==True)
mask4 = (comb["HomePlanet"] == "Earth")  & (comb["Destination"]=="TRAPPIST-1e") & (comb["CryoSleep"]==False)
mask5 = (comb["HomePlanet"] == "Earth")  & (comb["Cabin_1"]=="F") & (comb["CryoSleep"]==False)
mask6 = (comb["HomePlanet"] == "Earth")  & (comb["Cabin_3"]=="P") & (comb["CryoSleep"]==False)
mask7 = (comb["HomePlanet"] == "Europa")  & (comb["CryoSleep"]==True) & (comb["Cabin_3"]=="S")


In [37]:
comb["mask1"] = mask.astype(int)
comb["mask2"] = mask2.astype(int)
comb["mask3"] = mask3.astype(int)
comb["mask4"] = mask4.astype(int)
comb["mask5"] = mask5.astype(int)
comb["mask6"] = mask6.astype(int)
comb["mask7"] = mask7.astype(int)

In [38]:
comb.CryoSleep = comb.CryoSleep.astype(int)

In [39]:
comb.drop("VIP",axis=1,inplace=True)

In [40]:
comb.Cabin_3 = comb.Cabin_3.replace({"P":1,"S":0})

In [41]:
from category_encoders import OneHotEncoder
ohe = OneHotEncoder(
    cols=['Cabin_1','HomePlanet','Destination']

)
ohe.fit(comb)

OneHotEncoder(cols=['Cabin_1', 'HomePlanet', 'Destination'])

In [42]:
comb = ohe.transform(comb)

In [43]:
train = comb[comb['Transported'].notnull()].copy()
train.Transported =train.Transported.astype('int')
test = comb[comb['Transported'].isnull()].drop("Transported",axis=1)

In [44]:
X = train.drop('Transported',axis=1)
y = train.Transported

In [45]:
X,y = shuffle(X,y)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [72]:
# X.drop(["Cabin_1_1","Cabin_1_2","Cabin_1_5","Cabin_1_7","Cabin_1_8"],axis=1,inplace=True)

In [73]:
# X.drop(["HomePlanet_2","Destination_1","Destination_2","Destination_3"],axis=1,inplace=True)

In [58]:
# X.drop(["Spa","ShoppingMall","RoomService","FoodCourt","VRDeck"],axis=1,inplace=True)

In [58]:
X.drop(["Cabin_1_3","Cabin_1_8"],axis=1,inplace=True)

In [61]:
X.Cabin_1_1.value_counts()

Cabin_1_1
0    7897
1     796
Name: count, dtype: int64

In [ ]:
from xgboost import XGBClassifier
#params from another notebook
params= {'lambda': 3.0610042624477543, 
             'alpha': 4.581902571574289, 
             'colsample_bytree': 0.9241969052729379, 
             'subsample': 0.9527591724824661, 
             'learning_rate': 0.06672065863100594, 
             'n_estimators': 730,
             'max_depth': 5, 
             'min_child_weight': 1, 
             'num_parallel_tree': 1}

xgb = XGBClassifier(**params)

In [49]:
cross_val_score(xgb,X,y,cv=20,scoring="accuracy",n_jobs=-1)

0.8072045659198052


In [63]:
X

,ShoppingMall,FoodCourt,RoomService,Spa,VRDeck,Expenses,Age,inline,out,CryoSleep,Cabin_1_1,Cabin_1_2,Cabin_1_4,Cabin_1_5,Cabin_1_6,Cabin_1_7,Cabin_3,HomePlanet_1,HomePlanet_2,HomePlanet_3,Destination_1,Destination_2,Destination_3,mask1,mask2,mask3,mask4,mask5,mask6,mask7
0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,0,0,0
2,1.0,0.0,2424.0,0.0,284.0,2709.0,55.0,2708.0,1.0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
3,3.0,776.0,182.0,33.0,0.0,994.0,38.0,215.0,779.0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,55.0,0.0,740.0,85.0,880.0,27.0,825.0,55.0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0
8689,0.0,710.0,8.0,27.0,0.0,745.0,22.0,35.0,710.0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0
8690,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0
8691,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0


In [64]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
rf= RandomForestClassifier(n_estimators=600,max_depth=6)
sbs = SFS(estimator=rf, 
           k_features=10, 
           forward=False, 
           floating=True, 
           verbose=0,
           scoring='roc_auc',
           cv=20,
           n_jobs=-1)

sbs = sbs.fit(X, y)

pd.DataFrame.from_dict(sbs.get_metric_dict()).T.sort_values('avg_score',ascending=False)

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
16,"(3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, ...","[0.8800101471334348, 0.8805809233891425, 0.875...",0.88854,"(Spa, VRDeck, Expenses, Age, inline, out, Cryo...",0.006456,0.013492,0.003095
10,"(5, 7, 8, 9, 13, 15, 16, 17, 18, 20)","[0.8796507694909521, 0.8797881785895485, 0.876...",0.888426,"(Expenses, inline, out, CryoSleep, Cabin_1_5, ...",0.006124,0.012797,0.002936
11,"(5, 7, 8, 9, 11, 13, 15, 16, 17, 18, 20)","[0.8794288009470659, 0.8806549129037713, 0.877...",0.888223,"(Expenses, inline, out, CryoSleep, Cabin_1_2, ...",0.00605,0.012642,0.0029
17,"(3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, ...","[0.877283105022831, 0.8805280737358362, 0.8745...",0.887412,"(Spa, VRDeck, Expenses, Age, inline, out, Cryo...",0.006542,0.013671,0.003136
12,"(3, 5, 7, 8, 9, 11, 13, 15, 16, 17, 18, 20)","[0.879724759005581, 0.8817541856925418, 0.8746...",0.886953,"(Spa, Expenses, inline, out, CryoSleep, Cabin_...",0.006281,0.013125,0.003011
14,"(3, 4, 5, 7, 8, 9, 11, 13, 15, 16, 17, 18, 19,...","[0.8771456959242347, 0.8812362590901404, 0.874...",0.886917,"(Spa, VRDeck, Expenses, inline, out, CryoSleep...",0.006156,0.012863,0.002951
13,"(3, 4, 5, 7, 8, 9, 11, 13, 15, 16, 17, 18, 20)","[0.8768708777270421, 0.8836673431422291, 0.873...",0.886839,"(Spa, VRDeck, Expenses, inline, out, CryoSleep...",0.006155,0.012863,0.002951
18,"(3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, ...","[0.8764586504312533, 0.8807183324877388, 0.872...",0.886818,"(Spa, VRDeck, Expenses, Age, inline, out, Cryo...",0.006547,0.013681,0.003139
15,"(3, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17, 18,...","[0.8748203111787587, 0.8802215457466599, 0.873...",0.886537,"(Spa, VRDeck, Expenses, inline, out, CryoSleep...",0.006092,0.012729,0.00292
19,"(3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, ...","[0.8750317097919837, 0.8808557415863352, 0.872...",0.886174,"(Spa, VRDeck, Expenses, Age, inline, out, Cryo...",0.006524,0.013633,0.003128


In [101]:
pd.DataFrame.from_dict(sbs.get_metric_dict()).T.sort_values('avg_score',ascending=False)["feature_idx"].loc[10]

(5, 7, 8, 9, 13, 15, 16, 17, 18, 20)

In [94]:
x2 = X.iloc[:,[5, 7, 8, 9, 13, 15, 16, 17, 18, 20]]

In [89]:
x2

,Spa,VRDeck,Expenses,Age,inline,out,CryoSleep,Cabin_1_2,Cabin_1_4,Cabin_1_5,Cabin_1_7,Cabin_3,HomePlanet_1,HomePlanet_2,HomePlanet_3,Destination_1
0,0.0,0.0,0.0,40.0,0.0,0.0,0,0,0,0,0,1,1,0,0,1
1,0.0,0.0,0.0,48.0,0.0,0.0,1,0,0,0,0,1,1,0,0,0
2,0.0,284.0,2709.0,55.0,2708.0,1.0,0,0,0,0,0,0,0,0,1,1
3,33.0,0.0,994.0,38.0,215.0,779.0,0,0,0,1,0,0,0,1,0,1
4,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,740.0,85.0,880.0,27.0,825.0,55.0,0,1,0,0,0,1,0,1,0,1
8689,27.0,0.0,745.0,22.0,35.0,710.0,0,1,0,0,0,1,0,1,0,1
8690,0.0,0.0,0.0,15.0,0.0,0.0,1,0,1,0,0,1,0,1,0,1
8691,0.0,0.0,0.0,29.0,0.0,0.0,1,0,0,1,0,1,1,0,0,1


In [100]:
cross_val_score(xgb,x2,y,scoring="recall",n_jobs=-1).mean()

0.8419402478799739

In [77]:
test.drop(["Cabin_1_3","Cabin_1_8"],axis=1,inplace=True)

In [ ]:
def objective(trial):
    
    param = { 
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'lambda': trial.suggest_float('lambda', 0, 10.0),
        'alpha': trial.suggest_float('alpha', 0, 10.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1,1.0),
        'subsample': trial.suggest_float('subsample', 0.2,1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001,0.1),
        'n_estimators': trial.suggest_int('n_estimators', 400,1200),
        'max_depth': trial.suggest_categorical('max_depth', [2,3,4,5,6,7,8,9,10]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        
        }
    
    xgb = XGBClassifier(**param,tree_method='gpu_hist', gpu_id=0
        )

    score = cross_val_score(xgb, x2, y, n_jobs=-1, cv=20,scoring="roc_auc")
    roc_auc = score.mean()
    return roc_auc

study = optuna.create_study(pruner=optuna.pruners.HyperbandPruner(),
                                direction='maximize')
study.optimize(objective, n_trials=70)

In [ ]:
trial = study.best_trial
print(trial.value)
print(trial.params)

In [ ]:
xgb2 = XGBClassifier(**trial.params)

In [93]:
test2 = test.iloc[:,[5, 7, 8, 9, 13, 15, 16, 17, 18, 20]]

In [97]:
y_pred_proba = (xgb2.fit(x2,y)).predict_proba(test2)[::,1]
threshold = 0.48
y_pred = (y_pred_proba > threshold).astype('float')
sample['Transported'] = y_pred
sample['Transported']=sample['Transported']>0.5
sample.to_csv('5th.csv', index=False)